# Doppler Solve: Two Components

## Setup

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry
from pathlib import Path

starry_path = Path(starry.__file__).parents[0]
starry.config.lazy = True
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import george
import pymc3 as pm
import pymc3_ext as pmx
import theano.tensor as tt
from tqdm.auto import tqdm

## Generate

In [ ]:
# Settings
flux_err = 1e-4
ydeg = 15
nt = 16
inc = 60
veq = 40000
wav = np.linspace(642.85, 643.15, 200)
wav0 = np.linspace(642.75, 643.25, 200)

In [ ]:
# True intensity ratio (spot / photosphere)
ratio = 0.5

# True spectra (photosphere and spot)
spectrum1 = (
    1.0
    - 0.725 * np.exp(-0.5 * (wav0 - 643.0) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 642.895) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 642.97) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 643.1) ** 2 / 0.0085 ** 2)
)
spectrum2 = (
    1.0
    - 0.53 * np.exp(-0.5 * (wav0 - 642.93) ** 2 / 0.0085 ** 2)
    - 0.5 * np.exp(-0.5 * (wav0 - 643.05) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 642.895) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 642.97) ** 2 / 0.0085 ** 2)
    - 0.02 * np.exp(-0.5 * (wav0 - 643.1) ** 2 / 0.0085 ** 2)
)

# Prior on the spectra
spectral_mean1 = np.ones_like(wav0)
spectral_mean2 = np.ones_like(wav0)
spectral_cov1 = 1e-3 * np.ones_like(wav0)
spectral_cov2 = 1e-3 * np.ones_like(wav0)

# Plot them
fig, ax = plt.subplots(2, figsize=(12, 6), sharex=True, sharey=True)
ax[0].plot(wav0, spectrum1, "k-", label="true")
ax[0].plot(wav0, spectral_mean1, "C0-", label="prior")
ax[0].fill_between(
    wav0,
    spectral_mean1 - np.sqrt(spectral_cov1),
    spectral_mean1 + np.sqrt(spectral_cov1),
    color="C0",
    alpha=0.3,
)
ax[0].legend()
ax[1].plot(wav0, spectrum2, "k-", label="true")
ax[1].plot(wav0, spectral_mean2, "C1-", label="prior")
ax[1].fill_between(
    wav0,
    spectral_mean2 - np.sqrt(spectral_cov2),
    spectral_mean2 + np.sqrt(spectral_cov2),
    color="C1",
    alpha=0.3,
)
ax[1].legend()
ax[1].set_xlabel("rest wavelength [nm]")
ax[0].set_ylabel("spectrum 1")
ax[1].set_ylabel("spectrum 2");

In [ ]:
# Maps
image1 = np.mean(
    np.flipud(plt.imread(starry_path / "img" / "spot.png"))[:, :, :3], axis=2
)
image2 = 1 - image1

# Plot them
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image1, origin="lower", cmap="plasma", vmin=0, vmax=1)
im = ax[1].imshow(image2, origin="lower", cmap="plasma", vmin=0, vmax=1)
for axis in ax:
    axis.set_xticks([])
    axis.set_yticks([])
ax[0].set_title("map 1")
ax[1].set_title("map 2")
plt.colorbar(im, ax=ax, shrink=0.55);

In [ ]:
# Instantiate
map = starry.DopplerMap(
    ydeg=ydeg,
    nc=2,
    veq=veq,
    inc=inc,
    nt=nt,
    wav=wav,
    wav0=wav0,
    lazy=False,
    vsini_max=40000,
)
map.load(
    maps=[image1, image2],
    spectra=[spectrum1, ratio * spectrum2],
    smoothing=0.075,
)
map.show()

In [ ]:
# Generate the dataset
flux = map.flux(normalize=True)
flux += flux_err * np.random.randn(*flux.shape)

# Plot it
plt.figure(figsize=(3, 6))
plt.plot(
    wav, flux.T + np.linspace(0, 1, map.nt).reshape(1, -1), color="k", lw=1
)
plt.xlabel("wavelength [nm]")
plt.ylabel("intensity");

## Solve: Uniform prior

In [ ]:
with pm.Model() as model:

    # Instantiate a uniform map
    map = starry.DopplerMap(
        ydeg=ydeg,
        nc=2,
        veq=veq,
        inc=inc,
        nt=nt,
        wav=wav,
        wav0=wav0,
        lazy=True,
        vsini_max=40000,
    )

    # SHT matrix: converts from pixels to Ylms
    A = map.sht_matrix(smoothing=0.075)
    npix = A.shape[1]

    # Prior on the maps
    p = pm.Uniform("p", lower=0.0, upper=1.0, shape=(npix,))
    amp = pm.Uniform("amp", lower=0.0, upper=1.0)
    y1 = amp * tt.dot(A, p)
    y2 = amp * tt.dot(A, (1 - p))
    map._y = tt.concatenate(
        (tt.reshape(y1, (-1, 1)), tt.reshape(y2, (-1, 1))), axis=1
    )

    # Prior on the intensity ratio
    r = pm.Uniform("r", lower=0.0, upper=1.0)

    # Prior on the spectra
    np.random.seed(0)
    spectrum1 = pm.Bound(pm.Normal, upper=1.0)(
        "spectrum1",
        mu=spectral_mean1,
        sigma=np.sqrt(spectral_cov1),
        shape=(map.nw0,),
        testval=1 - np.sqrt(spectral_cov1) * np.abs(np.random.randn(map.nw0)),
    )
    spectrum2 = pm.Bound(pm.Normal, upper=1.0)(
        "spectrum2",
        mu=spectral_mean2,
        sigma=np.sqrt(spectral_cov2),
        shape=(map.nw0,),
        testval=1 - np.sqrt(spectral_cov1) * np.abs(np.random.randn(map.nw0)),
    )
    map.spectrum = tt.concatenate(
        (tt.reshape(spectrum1, (1, -1)), r * tt.reshape(spectrum2, (1, -1))),
        axis=0,
    )

    # Compute the model
    flux_model = map.flux()

    # Likelihood term
    pm.Normal(
        "obs",
        mu=tt.reshape(flux_model, (-1,)),
        sd=flux_err,
        observed=flux.reshape(
            -1,
        ),
    )

In [ ]:
niter = 2 * 50000
lr = 1e-4

loss = []
best_loss = np.inf
map_soln = model.test_point
with model:
    for obj, point in tqdm(
        pmx.optim.optimize_iterator(
            pmx.optim.Adam(lr=lr), niter, start=map_soln
        ),
        total=niter,
    ):
        loss.append(obj)
        if obj < best_loss:
            best_loss = obj
            map_soln = point

In [ ]:
loss = np.array(loss)
logloss = np.log10(loss)
logloss[loss < 0] = -np.log10(-loss[loss < 0])
plt.plot(np.arange(len(loss)), logloss, lw=1);

In [ ]:
with model:
    map.show_components(point=map_soln);

In [ ]:
with model:
    map.show(point=map_soln)